In [ ]:
# hourly script
# go to db, find all hourly tempo, unprocessed, send, register as processed

In [ ]:
# daily script
# go to db, find all hourly tempo, unprocessed, send, register as processed

In [ ]:
# for each channel
# find all subscribers
# title as spreadsheet tab
# write to excel
# send email with excel attachment

In [ ]:
# TODO:
# support other email service provider
# if too big, upload to s3, send s3 link

In [41]:
import shutil

In [67]:
import os
import yaml
import sqlalchemy
import pandas as pd
import uuid
import opends
import opends.easymail

In [78]:
import datetime
timestamp = datetime.datetime.utcnow().isoformat().split('.')[0]

In [72]:
config_file = os.environ['HOME']+'/.keys/scube_alerter.key.yaml'
with open(config_file, 'r') as fin:
	configs = yaml.load(fin, Loader=yaml.SafeLoader)
	conn_str       = configs['conn_str']
	slack_token    = configs['slack_token']
	sendgrid_token = configs['sendgrid_token']
# end with

In [79]:
sender_email = 'admin@i-scube.com'

In [73]:
engine = sqlalchemy.create_engine(conn_str)

In [8]:
conn = engine.connect()

In [9]:
cmd = 'SELECT * FROM alerts_backend WHERE _isProcessed=0'
df = pd.read_sql_query(cmd, con = conn)

In [10]:
df

,_uuid,title,msg,channel,_updated_at,_IsProcessed,_processed_at
0,153d77ec-dbec-4056-ac43-cb38de16f463,test,this is a test,generalalerts,1.563072e+09,0,-1.0
1,873ad105-b11a-40d6-b910-6a86cfb238a1,test,a test message,generalalerts,1.563083e+09,0,-1.0


In [31]:
df['timestamp'] = [pd.Timestamp.fromtimestamp(val).isoformat()+'Z' for val in df.['_updated_at']]

In [35]:
df = df[['timestamp', 'title', 'msg']].sort_values(by='timestamp')

In [36]:
DFs = {}

In [37]:
for title in df['title'].unique():
    DFs[title] = df[df['title'] == title]

In [20]:
filename = '/tmp/'+str(uuid.uuid4())+'.xlsx'

In [38]:
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

In [39]:
for title in DFs.keys():
    df = DFs[title]
    df.to_excel(writer, sheet_name=title)
# end for
writer.save()

In [40]:
filename

'/tmp/44e089f0-deb6-4b13-b0e6-2f3cc004fe14.xlsx'

In [42]:
# read mailing list

In [ ]:
+-------------+--------------+------+-----+---------+-------+
| Field       | Type         | Null | Key | Default | Extra |
+-------------+--------------+------+-----+---------+-------+
| _uuid       | varchar(40)  | NO   | PRI | NULL    |       |
| username    | text         | YES  |     | NULL    |       |
| _created_at | double       | YES  |     | NULL    |       |
| status      | varchar(20)  | YES  |     | NULL    |       |
| email       | varchar(320) | YES  |     | NULL    |       |
| channel     | text         | YES  |     | NULL    |       |
+-------------+--------------+------+-----+---------+-------+


In [53]:
cmd = "SELECT * FROM alerts_backend_subscription WHERE channel='{channel}' AND status='active'"
cmd = cmd.format(channel='generalalerts')

In [54]:
cmd

"SELECT * FROM alerts_backend_subscription WHERE channel='generalalerts' AND status='active'"

In [55]:
df_subscription = pd.read_sql_query(cmd, con=conn)

In [63]:
targets = list(df_subscription.transpose().to_dict().values())

In [64]:
targets

[{'_uuid': 'a4174c76-1ebd-42bd-acaa-f439edebdb30',
  'username': 'kuanpern',
  '_created_at': 1563104107.0,
  'status': 'active',
  'email': 'kptan86@gmail.com',
  'channel': 'generalalerts'}]

In [65]:
filename

'/tmp/44e089f0-deb6-4b13-b0e6-2f3cc004fe14.xlsx'

In [69]:
item = targets[0]

In [108]:
_email = item['email']
username = item['username']

In [86]:
tempo = 'hourly'
channel = 'generalalerts'

In [111]:
subject = '[{tempo}-alerts] {channel} {timestamp}'.format(
    tempo   = tempo,
    channel = channel,
    timestamp = timestamp,
) # end subject

In [109]:
text = 'Hi {username}, please see attachment for alerts.'.format(username=username)
text

'Hi kuanpern, please see attachment for alerts.'

In [110]:
opends.easymail.send_email(
    emailUser = sender_email,
    recipient = _email,
    api_key = sendgrid_token,
    subject = subject,
    text = text,
    attachmentFilePaths = [filename]
) # end send email